In [1]:
#Importing the positive and Negative words from Loughran and McDonald Dictionary

def loadPositive():


    myfile= open('/apps/apps1/a625533/Positive_words .csv',"r")
    positives = myfile.readlines()
    positive = [pos.strip().lower() for pos in positives]
#     print(positive)
    return positive

In [2]:
def loadNegative():


    yfile= open('/apps/apps1/a625533/Negative_words .csv',"r")
    negatives = yfile.readlines()
    negative = [neg.strip().lower() for neg in negatives]
# print(negative)

    return negative

In [3]:
#Function to get the cosine similarity

In [4]:
def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

In [5]:
#Function to count the number of words in the text

In [6]:
import re, math

WORD = re.compile(r'\w+')
    
def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

In [7]:
#Funtion to parse out MDA Section which is present in  Item 7 of a SEC Filing

def parse_mda(text, start=0):
    debug = False
#     """Parse normalized text 
#     """

    mda = ""
    end = 0
#     """
#         Parsing Rules
#     """

    # Define start & end signal for parsing
    item7_begins = [
        '\nItem 7.', '\nItem 7 –', '\nItem 7:', '\nItem 7 ', '\nItem 7\n' 
    ]
    item7_ends = ['\nItem 7A']
    if start != 0:
        item7_ends.append('\nItem 7')  # Case: ITEM 7A does not exist
    item8_begins = ['\nItem 8']
#     """
#         Parsing code section
#     """
    text = text[start:]

    # Get begin
    for item7 in item7_begins:
        begin = text.find(item7)
        if debug:
            print(item7, begin)
        if begin != -1:
            break

    if begin != -1:  # Begin found
        for item7A in item7_ends:
            end = text.find(item7A, begin + 1)
            if debug:
                print(item7A, end)
            if end != -1:
                break

        if end == -1:  # ITEM 7A does not exist
            for item8 in item8_begins:
                end = text.find(item8, begin + 1)
                if debug:
                    print(item8, end)
                if end != -1:
                    break

        # Get MDA
        if end > begin:
            mda = text[begin:end].strip()
        else:
            end = 0

    return mda


In [8]:
#Funtion to parse out Risk Factor Section which is present in  Item 1A of a SEC Filing

def parse_rf(text, start=0):
    debug = False
#     """Parse normalized text 
#     """

    rf = ""
    end = 0
#     """
#         Parsing Rules
#     """

    # Define start & end signal for parsing
    item1_begins = [
        '\nItem 1A.', '\nItem 1A –', '\nItem 1A:', '\nItem 1A ', '\nItem 1A\n' 
    ]
    item1_ends = ['\nItem 1B']
    if start != 0:
        item1_ends.append('\nItem 1A')  # Case: ITEM 7A does not exist
    item2_begins = ['\nItem 2']
#     """
#         Parsing code section
#     """
    text = text[start:]

    # Get begin
    for item1 in item1_begins:
        begin = text.find(item1)
        if debug:
            print(item1, begin)
        if begin != -1:
            break

    if begin != -1:  # Begin found
        for item1A in item1_ends:
            end = text.find(item1A, begin + 1)
            if debug:
                print(item1A, end)
            if end != -1:
                break

        if end == -1:  # ITEM 1A does not exist
            for item2 in item2_begins:
                end = text.find(item2, begin + 1)
                if debug:
                    print(item2, end)
                if end != -1:
                    break

        # Get Risk Factor
        if end > begin:
            rf = text[begin:end].strip()
        else:
            end = 0

    return rf

In [9]:
#Function to group the date to date stock price based on year (June-May)

def grouping(start_year,date_year,date_month,date_day):
    if date_month<6:
        group=date_year-start_year
    else:
        group=date_year-start_year+1
    return group

In [10]:
#Function to group as per the quarter

def group_quarter(start_year,date_year,date_month,date_day):
    if(date_month<3):
        group=(date_year-start_year)*4 - 1
    elif date_month>=3 and date_month<6:
        group=(date_year - start_year)*4
    elif date_month>=6 and date_month<9:
        group= (date_year - start_year)*4 + 1
    elif date_month>=9 and date_month<12:
        group=  (date_year - start_year)*4 + 2
    else:
        group= (date_year - start_year)*4 + 3
    return group

In [11]:
#Function which says which quarter of that year the particular date lies

def Number_quarter(start_year,date_year,date_month,date_day):
    if(date_month==3):
        Quarter_Number="Q4"
    elif date_month==12:
        Quarter_Number="Q3"
    elif date_month==6:
        Quarter_Number="Q1"
#     elif date_month==9:
#         Quarter_Number="Q2"
    else:
        Quarter_Number="Q2"
    return Quarter_Number

In [12]:
#The defination of year as per us is June of a particular year to May of the subsequent year . This function helps to categorize the stock price of the last quarter under the previous year . 
#Suppose for the year 2016
#the four quarter are June'16-Aug'16 , Sept'16-Nov'16 ,Dec'16-Feb'17 ,Mar'17-May'17
#This function helps us categorize Mar'17-May'17 also under the year 2016 .

def Year(start_year,date_year,date_month,date_day):
    if(date_month==3):
        year=date_year-1
    else:
        year=date_year
    return year

In [15]:
import edgar
from __future__ import division, print_function
import pandas as pd
from datetime import timedelta
import re, math
from collections import Counter
import numpy as np
import requests  # functions for interacting with web pages
from bs4 import BeautifulSoup  # DOM html manipulation
from pyparsing import (makeHTMLTags, SkipTo, commonHTMLEntity, replaceHTMLEntity, 
    htmlComment, anyOpenTag, anyCloseTag, LineEnd, OneOrMore, replaceWith)
from pyparsing import ParserElement
import unicodedata



#we are taking the company ticker ,company name and the central index number as input 
def edgar_data(Company_ticker, CompanyName, CIK_Number):
        #Extracting 10-k Documents of a company
        company = edgar.Company(CompanyName, CIK_Number)
        tree = company.getAllFilings(filingType = "10-K")
        # tree = company.getFilingsUrl(filingType = "10-K")
        docs = edgar.getDocuments(tree, noOfDocuments=8)
        
        #We are extracting edgar filing of the company for the last 8 years


        df = pd.DataFrame({'text':docs})


        #Formatting the whole Text

        FormatText=[]
        for text in docs:

            text = unicodedata.normalize("NFKD", text)  # Normalize
            text = '\n'.join(
                text.splitlines())  # Let python take care of unicode break lines

            # Convert to upper
        #     text = text.upper()  # Convert to upper

            # Take care of breaklines & whitespaces combinations due to beautifulsoup parsing
            text = re.sub(r'[ ]+\n', '\n', text)
            text = re.sub(r'\n[ ]+', '\n', text)
            text = re.sub(r'\n+', '\n', text)

            # To find MDA section, reformat item headers
            text = text.replace('\n.\n', ' ')  # Move Period to beginning

        # #     text = text.replace('\nI\nTEM', '\nITEM')
        #     text = text.replace('\nI\ntem', '\nItem')
        # #     text = text.replace('\nITEM\n', '\nITEM ')
        #     text= text.replace('\nItem\n' ,'\nItem')
        # #     text = text.replace('\nITEM  ', '\nITEM ')
        #     text= text.replace('\nItem' , '\nItem')

            text = text.replace(':\n', ' ')

            # Math symbols for clearer looks
            text = text.replace('$\n', '$')
            text = text.replace('\n%', '%')
            text = text.replace('\n', ' ')

        #     # Reformat
        #     text = text.replace('\n', '\n\n')  # Reformat by additional breakline

            FormatText.append(text)

        df['format_text']=FormatText

        #Find out Jaccard Similarity for the entire document

        jaccard_Whole_Text=['NA']

        str=[FormatText[0],FormatText[1],FormatText[2],FormatText[3],FormatText[4],FormatText[5],FormatText[6],FormatText[7]]
        for i in range(0,7):

            a = set(str[i%8].split())
            b = set(str[(i+1)%8].split())
            c = a.intersection(b)
            sim_jaccard_method= float(len(c)) / (len(a) + len(b) - len(c))
            jaccard_Whole_Text.append(sim_jaccard_method)

        df['jaccard_similarity_whole_text ']=jaccard_Whole_Text

        

        WORD = re.compile(r'\w+')



        #Cosine Similarity of the entire document for two consecutive years

        Cosine_FullText=['NA']
        str=[FormatText[0],FormatText[1],FormatText[2],FormatText[3],FormatText[4],FormatText[5],FormatText[6],FormatText[7]]

        for i in range(0,7):


            vector1 = text_to_vector(str[i%8])
            vector2 = text_to_vector(str[(i+1)%8])

            cosine = get_cosine(vector1, vector2)
            Cosine_FullText.append(cosine)

        df['cosine_similarity_whole_text']=Cosine_FullText

        #Finding out positive and Negative words in the document

        Count_positive=[]
        Count_negative=[]
        diff=[]
        for text in FormatText:


            
            positive_number=loadPositive()
            negative_number=loadNegative()
            
            # count = Counter(paragraph.split())
            count = Counter(text.split())
            #print(count.items())

            pos = 0
            neg = 0
            for key, val in count.items():
                key = key.rstrip('.,?!\n') # removing possible punctuation signs
                if key in positive_number:
                    pos += val
                if key in negative_number:
                    neg += val
            Difference=neg-pos
            Count_positive.append(pos)
            Count_negative.append(neg)
            diff.append(Difference)

        df['count_positive_fulltext']=Count_positive
        df['count_negative_fulltext']=Count_negative
        df['difference_fulltext']=diff

        #To find total number of words in the document

        Total_words=[]
        for fname in FormatText:

            num_words = 0

            for line in fname:
                words = line.split()
                num_words += len(words)
            Total_words.append(num_words)

        df['total_words_fulltext']=Total_words

        #To find out pessimism = (Negative-Positive)/Total number of words

        df['pessimism']=df['difference_fulltext']/df['total_words_fulltext']

        #Change in Pessimism over last year

        df['change_in_pessimism_over_last_year'] = df['pessimism'] - df['pessimism'].shift(-1)

        #To parse out MDA

        NewText=[]

        for index, row in df.iterrows():

        #             start = 'Changes in and Disagreements with Accountants on Accounting and Financial Disclosure'
#                     start = 'Changes in and Disagreements with Accountants on Accounting and Financial'
#                     start = 'Changes in and Disagreements with Accountants on'
                    start = 'Changes in and Disagreements'
#                     start = 'Disagreements with'
                    s = row['text']
                    new1=s.split(start)[1]
        #             print(new1)
                    NewText.append(new1)

        df['new_text']=NewText

        #Formatting the text to remove new line

        FormatText_New=[]
        for text in NewText:

            text = unicodedata.normalize("NFKD", text)  # Normalize
            text = '\n'.join(
                text.splitlines())  # Let python take care of unicode break lines

            # Convert to upper
        #     text = text.upper()  # Convert to upper

            # Take care of breaklines & whitespaces combinations due to beautifulsoup parsing
            text = re.sub(r'[ ]+\n', '\n', text)
            text = re.sub(r'\n[ ]+', '\n', text)
            text = re.sub(r'\n+', '\n', text)

            # To find MDA section, reformat item headers
            text = text.replace('\n.\n', '.\n')  # Move Period to beginning

        #     text = text.replace('\nI\nTEM', '\nITEM')
            text = text.replace('\nI\ntem', '\nItem')
        #     text = text.replace('\nITEM\n', '\nITEM ')
            text= text.replace('\nItem\n' ,'\nItem')
        #     text = text.replace('\nITEM  ', '\nITEM ')
            text= text.replace('\nItem' , '\nItem')

            text = text.replace(':\n', '.\n')

            # Math symbols for clearer looks
            text = text.replace('$\n', '$')
            text = text.replace('\n%', '%')

            # Reformat
            text = text.replace('\n', '\n\n')  # Reformat by additional breakline

            FormatText_New.append(text)

        mda_list=[]
        for text in FormatText_New:
            mda = parse_mda(text)
            mda_list.append(mda)
        #     print(type(mda))
            # Parse second time if first parse results in index
            if mda and len(mda.encode('utf-8')) < 1000:
                mda= parse_mda(text, start=end)

        #Format MDA part

        MDA_FormatText=[]
        for text in mda_list:

            text = unicodedata.normalize("NFKD", text)  # Normalize
            text = '\n'.join(
                text.splitlines())  # Let python take care of unicode break lines

            # Convert to upper
        #     text = text.upper()  # Convert to upper

            # Take care of breaklines & whitespaces combinations due to beautifulsoup parsing
            text = re.sub(r'[ ]+\n', '\n', text)
            text = re.sub(r'\n[ ]+', '\n', text)
            text = re.sub(r'\n+', '\n', text)

            # To find MDA section, reformat item headers
            text = text.replace('\n.\n', ' ')  # Move Period to beginning

        # #     text = text.replace('\nI\nTEM', '\nITEM')
        #     text = text.replace('\nI\ntem', '\nItem')
        # #     text = text.replace('\nITEM\n', '\nITEM ')
        #     text= text.replace('\nItem\n' ,'\nItem')
        # #     text = text.replace('\nITEM  ', '\nITEM ')
        #     text= text.replace('\nItem' , '\nItem')

            text = text.replace(':\n', ' ')

            # Math symbols for clearer looks
            text = text.replace('$\n', '$')
            text = text.replace('\n%', '%')
            text = text.replace('\n', ' ')

        #     # Reformat
        #     text = text.replace('\n', '\n\n')  # Reformat by additional breakline

            MDA_FormatText.append(text)

        df['mda_formattext']=MDA_FormatText

        #Find Jaccard Similarity for the formatted MDA Section

        #Formatted MDA Jaccard Similarity
        jaccard_MDA=['NA']

        str=[MDA_FormatText[0],MDA_FormatText[1],MDA_FormatText[2],MDA_FormatText[3],MDA_FormatText[4],MDA_FormatText[5],MDA_FormatText[6],MDA_FormatText[7]]
        for i in range(0,7):

            a = set(str[i%8].split())
            b = set(str[(i+1)%8].split())
            c = a.intersection(b)
            sim_jaccard_method= float(len(c)) / (len(a) + len(b) - len(c))
            jaccard_MDA.append(sim_jaccard_method)
        #     print(sim_jaccard_method)

        df['jaccard_similarity_mda']=jaccard_MDA

        #To find out Cosine Similarity

        #Cosine SImilarity MDA Section
        Cosine_MDA=['NA']
        str=[MDA_FormatText[0],MDA_FormatText[1],MDA_FormatText[2],MDA_FormatText[3],MDA_FormatText[4],MDA_FormatText[5],MDA_FormatText[6],MDA_FormatText[7]]

        for i in range(0,7):


            vector1 = text_to_vector(str[i%8])
            vector2 = text_to_vector(str[(i+1)%8])

            cosine = get_cosine(vector1, vector2)
            Cosine_MDA.append(cosine)

        df['cosine_similarity_mda']=Cosine_MDA

        #To find out Negative and Positive word in MDA Section

        Count_positive_MDA=[]
        Count_negative_MDA=[]
        diff_MDA=[]
        for text in MDA_FormatText:


            positive_number=loadPositive()
            negative_number=loadNegative()
            # count = Counter(paragraph.split())
            count = Counter(text.split())
            #print(count.items())

            pos = 0
            neg = 0
            for key, val in count.items():
                key = key.rstrip('.,?!\n') # removing possible punctuation signs
                if key in positive_number:
                    pos += val
                if key in negative_number:
                    neg += val
            Difference_MDA=neg-pos
            Count_positive_MDA.append(pos)
            Count_negative_MDA.append(neg)
            diff_MDA.append(Difference_MDA)

        df['count_positive_mda']=Count_positive_MDA
        df['count_negative_mda']=Count_negative_MDA
        df['difference_mda']=diff_MDA

        #To find out total number of words in MDA Section

        Total_words_MDA=[]
        for fname in MDA_FormatText:

            num_words = 0

            for line in fname:
                words = line.split()
                num_words += len(words)
            Total_words_MDA.append(num_words)

        df['total_words_mda']=Total_words_MDA

        df['pessimism_mda']=df['difference_mda']/df['total_words_mda']

        #Find out the change in Pessism

        df['change_in_pessimism_over_last_year_mda'] = df['pessimism_mda'] - df['pessimism_mda'].shift(-1)



        RiskFactor_list=[]
        for text in FormatText_New:
            rf = parse_rf(text)
            RiskFactor_list.append(rf)
        #     print(type(rf))
            # Parse second time if first parse results in index
            if rf and len(rf.encode('utf-8')) < 1000:
                rf= parse_rf(text, start=end)

        RiskFactor_FormatText=[]
        for text in RiskFactor_list:

            text = unicodedata.normalize("NFKD", text)  # Normalize
            text = '\n'.join(
                text.splitlines())  # Let python take care of unicode break lines

            # Convert to upper
        #     text = text.upper()  # Convert to upper

            # Take care of breaklines & whitespaces combinations due to beautifulsoup parsing
            text = re.sub(r'[ ]+\n', '\n', text)
            text = re.sub(r'\n[ ]+', '\n', text)
            text = re.sub(r'\n+', '\n', text)

            # To find MDA section, reformat item headers
            text = text.replace('\n.\n', ' ')  # Move Period to beginning

        # #     text = text.replace('\nI\nTEM', '\nITEM')
        #     text = text.replace('\nI\ntem', '\nItem')
        # #     text = text.replace('\nITEM\n', '\nITEM ')
        #     text= text.replace('\nItem\n' ,'\nItem')
        # #     text = text.replace('\nITEM  ', '\nITEM ')
        #     text= text.replace('\nItem' , '\nItem')

            text = text.replace(':\n', ' ')

            # Math symbols for clearer looks
            text = text.replace('$\n', '$')
            text = text.replace('\n%', '%')
            text = text.replace('\n', ' ')

        #     # Reformat
        #     text = text.replace('\n', '\n\n')  # Reformat by additional breakline

            RiskFactor_FormatText.append(text)

        df['riskfactor_formattext']=RiskFactor_FormatText

        #Find out Jaccard Similarity for Risk Factor Section
        jaccard_RF=['NA']

        str=[RiskFactor_list[0],RiskFactor_list[1],RiskFactor_list[2],RiskFactor_list[3],RiskFactor_list[4],RiskFactor_list[5],RiskFactor_list[6],RiskFactor_list[7]]
        for i in range(0,7):

            a = set(str[i%8].split())
            b = set(str[(i+1)%8].split())
            c = a.intersection(b)
            sim_jaccard_method= float(len(c)) / (len(a) + len(b) - len(c))
            jaccard_RF.append(sim_jaccard_method)
        #     print(sim_jaccard_method)


        df['jaccard_similarity_rf']=jaccard_RF

        #Cosine Similarity for Risk Factor Section

        Cosine_RF=['NA']
        str=[RiskFactor_FormatText[0],RiskFactor_FormatText[1],RiskFactor_FormatText[2],RiskFactor_FormatText[3],RiskFactor_FormatText[4],RiskFactor_FormatText[5],RiskFactor_FormatText[6],RiskFactor_FormatText[7]]

        for i in range(0,7):


            vector1 = text_to_vector(str[i%8])
            vector2 = text_to_vector(str[(i+1)%8])

            cosine = get_cosine(vector1, vector2)
            Cosine_RF.append(cosine)


        df['cosine_similarity_rf']=Cosine_RF

        #Find out Positive and Negative words in RiskFactor Section

        Count_positive_RF=[]
        Count_negative_RF=[]
        diff_RF=[]
        for text in RiskFactor_FormatText:


            pos_number=loadPositive()
            neg_number=loadNegative()
            # count = Counter(paragraph.split())
            count = Counter(text.split())
            #print(count.items())

            pos = 0
            neg = 0
            for key, val in count.items():
                key = key.rstrip('.,?!\n') # removing possible punctuation signs
                if key in pos_number:
                    pos += val
                if key in neg_number:
                    neg += val
            Difference_RF=neg-pos
            Count_positive_RF.append(pos)
            Count_negative_RF.append(neg)
            diff_RF.append(Difference_RF)

        df['count_positive_rf']=Count_positive_RF
        df['count_negative_rf']=Count_negative_RF
        df['difference_rf']=diff_RF

        #Find out total words in Risk Factor Section

        Total_words_RF=[]
        for fname in RiskFactor_FormatText:

            num_words = 0

            for line in fname:
                words = line.split()
                num_words += len(words)
            Total_words_RF.append(num_words)

        df['total_words_rf']=Total_words_RF

        df['pessimism_rf']=df['difference_rf']/df['total_words_rf']

        #Find the change in Pessimism in Risk Factor Section

        df['change_in_pessimism_over_last_year_rf'] = df['pessimism_rf'] - df['pessimism_rf'].shift(-1)

        #Importing the daily stock report from yahoo

        from pandas_datareader import data, wb
        import datetime

        # We will look at stock prices over the past year, starting at January 1, 2010
        start = datetime.datetime(2010,6,1)
        # end = datetime.date.today()
        end= datetime.datetime(2018,5, 30)

        # First argument is the series we want, second is the source ("yahoo" for Yahoo! Finance), third is the start date, fourth is the end date
        # Get the stock price data from yahoo for the company 
        Company_name = data.get_data_yahoo(Company_ticker, start, end) 

        df_new = pd.DataFrame(data=Company_name)  #Company_Name
        df_new=df_new.drop('Adj Close',axis=1)

        df_new=df_new.drop('Volume',axis=1)

        df_new=df_new.reset_index()
        
        #Year wise grouping and find out Open,close, high , low for that particular year(June-May)

        group=[]
        for value in range(df_new['Date'].shape[0]):
            curr_year=df_new['Date'].dt.year[value]
            curr_month=df_new['Date'].dt.month[value]
            curr_day=df_new['Date'].dt.day[value]
            group.append(grouping(2010,curr_year,curr_month,curr_day))

        df_new['Group Number']=group

        low=[]
        low_value=df_new.groupby("Group Number").min()["Low"].reset_index()
        low.append(low_value)


        low=low[::-1]

        high=[]
        high_value=df_new.groupby("Group Number").max()["High"].reset_index()
        high.append(high_value)

        high=high[::-1]

        df3=df_new.groupby('Group Number').first().reset_index()

        open= df3['Open']  #FIND THE OPEN VALUES FOR EACH GROUP

        open=open[::-1]

        date= df3['Date'] #start date of each quarter

        dflast=df_new.groupby('Group Number').last().reset_index()

        close= dflast['Close']

        close=close[::-1]

        date=date[::-1]

        df_yearly=pd.DataFrame({'date':date})
        df_yearly['open']=open
        df_yearly['high']=high_value['High']
        df_yearly['low']=low_value['Low']
        df_yearly['close']=close

        df_yearly.index = range(8)
        
        #Quarter wise grouping and find out open,close,high ,low for each quarter in that year (June-Aug, Sept-Nov, Dec-Feb ,Mar-May)

        df_quart = pd.DataFrame(data=Company_name)

        df_quart=df_quart.drop('Adj Close',axis=1)

        df_quart=df_quart.drop('Volume',axis=1)

        df_quart=df_quart.reset_index()

        group1=[]
        for value in range(df_quart['Date'].shape[0]):
            curr_year=df_quart['Date'].dt.year[value]
            curr_month=df_quart['Date'].dt.month[value]
            curr_day=df_quart['Date'].dt.day[value]
            group1.append(group_quarter(2010,curr_year,curr_month,curr_day))

        df_quart['Quarterly Group']=group1

        df_quart=df_quart.reset_index()

        low_Quarter=[]
        low_value=df_quart.groupby("Quarterly Group").min()["Low"].reset_index()
        low_Quarter.append(low_value)

        low_Quarter=low_Quarter[::-1]

        high_Quarter=[]
        high_value=df_quart.groupby("Quarterly Group").max()["High"].reset_index()
        high_Quarter.append(high_value)

        high_Quarter=high_Quarter[::-1]

        df6=df_quart.groupby('Quarterly Group').first().reset_index() 

        open= df6['Open']

        open=open[::-1]

        df7=df_quart.groupby('Quarterly Group').last().reset_index()

        date=df6['Date']
        date=date[::-1]
        close= df6['Close']
        close=close[::-1]

        close_value=[]
        close_value.append(close)

        start_date=[]
        start_date.append(date)

        df_quarterwise=pd.DataFrame({'date':date})

        df_quarterwise['open']=open

        df_quarterwise['high']=high_value['High']

        df_quarterwise['low']=low_value['Low']
        df_quarterwise['close']=close



        Number=[]
        for value in range(df_quarterwise['date'].shape[0]):
            curr_year=df_quarterwise['date'].dt.year[value]
            curr_month=df_quarterwise['date'].dt.month[value]
            curr_day=df_quarterwise['date'].dt.day[value]
            Number.append(Number_quarter(2010,curr_year,curr_month,curr_day))

        Number=Number[::-1]

        df_quarterwise['Quarter Number']=Number


        Year_Quarter=[]
        for value in range(df_quarterwise['date'].shape[0]):
            curr_year=df_quarterwise['date'].dt.year[value]
            curr_month=df_quarterwise['date'].dt.month[value]
            curr_day=df_quarterwise['date'].dt.day[value]
            Year_Quarter.append(Year(2010,curr_year,curr_month,curr_day))

        Year_Quarter=Year_Quarter[::-1]

        df_quarterwise['year']=Year_Quarter

        df_quarterwise=df_quarterwise.drop('date',axis=1)

        df_quarterwise.index=range(32)

        df_final=df_quarterwise.pivot_table(index=["year"][::-1],columns=["Quarter Number"],values=["open","high","low","close"])

        df_quarterfinal=df_final.swaplevel(0, 1, 1).sort_index(1)

        df_quarterfinal=df_quarterfinal[::-1]

        df_quarterfinal.index = range(8)
        
        #datafrane which concats the scrape out section of the SEC Filing and the analysis, yearly stock price (June-May), Quarter-wise stock price (June-Aug, Sept-Nov, Dec-Feb ,Mar-May)

        final=pd.concat([df,df_yearly,df_quarterfinal],axis=1)
        
        #Renaming the column name 

        fin_table=final.rename(columns={ final.columns[34]: "Q1_close",final.columns[35]:"Q1_high",final.columns[36]:"Q1_low", final.columns[37]:"Q1_open",final.columns[38]:"Q2_close",final.columns[39]:"Q2_high",final.columns[40]:"Q2_low",final.columns[41]:"Q2_open",final.columns[42]:"Q3_close",final.columns[43]:"Q3_high",final.columns[44]:"Q3_low",final.columns[45]:"Q3_open",final.columns[46]:"Q4_close",final.columns[47]:"Q4_high",final.columns[48]:"Q4_low",final.columns[49]:"Q4_open"})

        return fin_table
        # return the dataframe consiting of all the details for a company (Scraping section ,Finding cosine and jacarrd similarity, pessimism change as well as the stock price for that particular year )

In [53]:
r=edgar_data("NTAP","NetApp, Inc.","0001002047")

In [54]:
print(r)

                                                text  \
0  \n10-K\n1\nntap-10k_20180427.htm\n10-K\n\n\n\n...   
1  \n10-K\n1\nntap-10k_20170428.htm\n10-K\n\n\n\n...   
2  \n10-K\n1\nntap-10k_20160429.htm\n10-K\n\n\n\n...   
3  \n10-K\n1\nntap-10k_20150424.htm\n10-K\n\n\n\n...   
4  \n10-K\n1\nd691214d10k.htm\n10-K\n\n\n10-K\n\n...   
5  \n10-K\n1\nd512428d10k.htm\nFORM 10-K\n\n\nFor...   
6  \n10-K\n1\nd328654d10k.htm\nFORM 10-K\n\n\nFor...   
7  \n10-K\n1\nd10k.htm\nFORM 10-K\n\n\nForm 10-K\...   

                                         format_text  \
0   10-K 1 ntap-10k_20180427.htm 10-K ntap-10k_20...   
1   10-K 1 ntap-10k_20170428.htm 10-K ntap-10k_20...   
2   10-K 1 ntap-10k_20160429.htm 10-K ntap-10k_20...   
3   10-K 1 ntap-10k_20150424.htm 10-K ntap-10k_20...   
4   10-K 1 d691214d10k.htm 10-K 10-K Table of Con...   
5   10-K 1 d512428d10k.htm FORM 10-K Form 10-K Ta...   
6   10-K 1 d328654d10k.htm FORM 10-K Form 10-K Ta...   
7   10-K 1 d10k.htm FORM 10-K Form 10-K Table o

In [17]:
#Reading two excel sheet (One containing the list of tickers and the other containing the Ticker, Company Name and CIK Number)


import pandas as pd



df7=pd.read_excel('/apps/apps1/a625533/tick.xlsx')
df8=pd.read_excel('/apps/apps1/a625533/fin.xlsx')

In [18]:
#Finding out the company name and CIK number for the list of tickers for which we want the required datas

In [19]:
results=df8.merge(df7,on='Ticker')

In [20]:
results=results.drop('Unnamed: 2',axis=1)

In [21]:
results.head(20)

,Ticker,Name,CIK
0,AAOI,Applied Optoelectronics Inc,1158114
1,AKS,Ak Steel Holding Corp,918160
2,AMC,American Mortgage Acceptance Co,878774
3,AMC,Amc Entertainment Holdings Inc,1411579
4,AMD,Advanced Micro Devices Inc,2488
5,AMPE,American Petroleum Group Inc,1138593
6,AMPE,Ampio Pharmaceuticals Inc,1411906
7,AN,Autonation Inc,350698
8,ANCX,Access National Corp,1176316
9,AR,Antero Resources Corp,1433270


In [24]:
#Iterating over 50 tickers and checking for how many companies we are able to get result

In [22]:
c=0
for index, row in results.head(50).iterrows():
    
    Company_ticker=row['Ticker']
    Company_Name=row['Name']
    CIK_Number=str(row['CIK'])
    
    
    try:
    
        r=edgar_data(Company_ticker, Company_Name, CIK_Number)
        c+=1
        print(row['Ticker'])
    
    except:
        pass

CRR
DO


In [23]:
print(c)

2


In [55]:
rslt_df = results.loc[results['Ticker']=='CRR'] 

In [56]:
rslt_df

,Ticker,Name,CIK
33,CRR,Carbo Ceramics Inc,1009672


In [57]:
O=edgar_data("CRR","Carbo Ceramics Inc","1009672")

In [58]:
print(O)

                                                text  \
0  \n10-K\n1\ncrr-10k_20181231.htm\n10-K\n\n\n\n\...   
1  \n10-K\n1\ncrr-10k_20171231.htm\n10-K\n\n\n\n\...   
2  \n10-K\n1\ncrr-10k_20161231.htm\n10-K\n\n\n\n\...   
3  \n10-K\n1\nd104557d10k.htm\nFORM 10-K\n\n\nFor...   
4  \n10-K\n1\nd839115d10k.htm\n10-K\n\n\n10-K\n\n...   
5  \n10-K\n1\nd637538d10k.htm\n10-K\n\n\n10-K\n\n...   
6  \n10-K\n1\nd439385d10k.htm\nFORM 10-K\n\n\nFOR...   
7  \n10-K\n1\nd238954d10k.htm\nFORM 10-K\n\n\nFor...   

                                         format_text  \
0   10-K 1 crr-10k_20181231.htm 10-K crr-10k_2018...   
1   10-K 1 crr-10k_20171231.htm 10-K crr-10k_2017...   
2   10-K 1 crr-10k_20161231.htm 10-K crr-10k_2016...   
3   10-K 1 d104557d10k.htm FORM 10-K Form 10-K Ta...   
4   10-K 1 d839115d10k.htm 10-K 10-K UNITED STATE...   
5   10-K 1 d637538d10k.htm 10-K 10-K Table of Con...   
6   10-K 1 d439385d10k.htm FORM 10-K FORM 10-K Ta...   
7   10-K 1 d238954d10k.htm FORM 10-K Form 10-K 

In [ ]:
c=0
for index, row in results.iterrows():
    
    Company_ticker=row['Ticker']
    Company_Name=row['Name']
    CIK_Number=str(row['CIK'])
    
    
    try:
    
        r=edgar_data(Company_ticker, Company_Name, CIK_Number)
        c+=1
        print(row['Ticker'])
    
    except:
        pass

In [60]:
print(c)

3
